### This notebook serves the purpose of running everithing in one go - train, save the model where you want, resume training, see the predictions etc.


In [1]:
device = 'cuda:1'

In [2]:
from baseline_preprocess_input import *
from baseline_model import *
from base_train import *


In [3]:
# uncomment if you want to see the character keys from dict
# print(char2int.keys())

In [4]:
n_hidden=512
n_layers=2

model = CharRNN(chars, device, n_hidden, n_layers)
print(model)

CharRNN(
  (lstm): LSTM(224, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=224, bias=True)
)


In [12]:
batch_size = 128
seq_length = 300
n_epochs = 2 # small because for testing

# train the model
train_and_save(model, encoded, device, model_name='rnn_5_epoch.pt', epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.0001, resume_from_saved=False)

Starting to train rnn_5_epoch.pt...


RuntimeError: CUDA error: out of memory

In [10]:
opt = torch.optim.Adam(model.parameters(), lr=0.0001)

In [14]:
from predict import *

model, _ , _, _, _, _  = load_checkpoint("rnn_5_epoch.pt", model, opt)

show_sample(model, 100, device, prime='ja', top_k=5)

'ja  Soráligii ja dana mieldi da lit luvto miilgii gáilda dángit dui suhtiid deat lal  oahkke meahkkiis '